In [1]:
%load_ext dotenv
%dotenv

In [2]:
from VLA_LLM import prompts
from VLA_LLM.agents import ChatConversationalVLAAgent
from VLA_LLM.tools import AppointmentSchedulerAndAvailabilityTool
from VLA_LLM.tools import CurrentTimeTool
from VLA_LLM.community_info import EXAMPLE_COMMUNITY_DATA
from VLA_LLM.community_info import community_dict_to_prompt

In [3]:
tools = [CurrentTimeTool(), AppointmentSchedulerAndAvailabilityTool()]

In [19]:
# most deterministic results
temperature = 0.5

In [20]:
community_info = community_dict_to_prompt(EXAMPLE_COMMUNITY_DATA)

In [21]:
print(community_info)

Parking options:
  1. Parking option 1
     Parking type: Parking Available
     Parking fee: $100.0
     Parking fee term: Monthly
     Parking notes: noties parking available



In [22]:
prompt_template = f"{prompts.prompt_2.format(community_info=community_info)}\n\nHere is the prospect message:\n\n{{prospect_message}}"

In [23]:
print(prompt_template)

You are a leasing agent at a large multifamily apartment building talking with a prospect interested in renting one of your units. Your job is to answer questions about the community based on the information below. You should also try to gather their preferences like budget, move-in date, and desired layout, while nudging them to book a tour.

Here is information about the property that you can answer questions about:

Parking options:
  1. Parking option 1
     Parking type: Parking Available
     Parking fee: $100.0
     Parking fee term: Monthly
     Parking notes: noties parking available


If the prospect provides an appointment time or date, first call the get_current_time tool to get the current day of week and time. Then pass the appointment time or date to the appointment scheduler tool in paraphrase the result to the prospect conversationally.

If they do not provide an appointment time or date, do not call the tool.

*Note*: Appointment times should always be interpreted wit

In [24]:
agent = ChatConversationalVLAAgent(temperature, tools)

In [30]:
def run_agent(message):
    return agent.agent_chain.run(input=prompt_template.format(prospect_message=message))

### Test Conversation 1

Basic question about the community.

In [31]:
run_agent("What is the parking situation?")



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "The parking options at our community include Parking option 1. It is a parking available option with a monthly fee of $100.0. Please let me know if you have any other questions or if there's anything else I can assist you with!"
}

> Finished chain.


"The parking options at our community include Parking option 1. It is a parking available option with a monthly fee of $100.0. Please let me know if you have any other questions or if there's anything else I can assist you with!"

### Test Conversation 2

Asking about an available appointment date.

In [32]:
run_agent("Can I schedule a tour for 7/26?")



> Entering new AgentExecutor chain...
{
    "action": "get_current_time",
    "action_input": ""
}
Observation: Tuesday 2023-07-25 21:33:48
Thought:{
    "action": "appointment_scheduler_availability",
    "action_input": "7/26"
}
Observation: Appointment date is not in the correct format. The agent should provide it in YYYY-MM-DD format, then run again.
Thought:{
    "action": "Final Answer",
    "action_input": "I apologize for the inconvenience. Could you please provide the desired date for the tour in the format YYYY-MM-DD? Once I have that information, I can check the availability and schedule the tour for you."
}

> Finished chain.


'I apologize for the inconvenience. Could you please provide the desired date for the tour in the format YYYY-MM-DD? Once I have that information, I can check the availability and schedule the tour for you.'

### Test Conversation 3

Asking about an unavailable appointment date.

In [33]:
run_agent("You do have any available appointments on 7/28?")



> Entering new AgentExecutor chain...
{
    "action": "appointment_scheduler_availability",
    "action_input": "7/28"
}
Observation: Appointment date is not in the correct format. The agent should provide it in YYYY-MM-DD format, then run again.
Thought:{
    "action": "Final Answer",
    "action_input": "I apologize for the inconvenience. Could you please provide the desired date for the tour in the format YYYY-MM-DD? Once I have that information, I can check the availability and schedule the tour for you."
}

> Finished chain.


'I apologize for the inconvenience. Could you please provide the desired date for the tour in the format YYYY-MM-DD? Once I have that information, I can check the availability and schedule the tour for you.'